In [1]:
#------------------------------------------------------
#Data set
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
#Forward propagation
from decimal import Decimal, getcontext
import math
def forward_prop():
    global X,h,O,w_hidden,w_output,b_hidden,b_output

    #calculate hidden layer values
    for i in range(5):
        val=0.0
        for j in range(4):
            val=val+(X[j]*w_hidden[j][i])
        #using relu on hidden layer f(x)=(0,x)
        h[i]=max(0,val+b_hidden[i])


    #calculate output layer values
    for i in range(3):
        val=0.0
        for j in range(5):
            val=val+(h[j]*w_output[j][i])
        O[i]=val+b_output[i]

    # Set the precision for Decimal calculations
    getcontext().prec = 30

    max_O = max(O)
    sum_decimal = Decimal('0.0')
    exp_decimal = []

    # Compute the exponential of each value and sum them
    for i in range(3):
        exp_decimal.append(Decimal(math.exp(O[i]- max_O)))
        sum_decimal += exp_decimal[i]

    for i in range(3):
        O[i] = exp_decimal[i] / sum_decimal

    # Convert Decimal values to floats
    O = [float(x) for x in O]


In [62]:
#For Error calculation
def abs_error():
  global E,actual_output,O
  for i in range(3):
      E[i]=abs(actual_output[i]-O[i])

In [63]:
#-----------------Back propagation------------------------
#calculating delta_O,delta_h
def func1():
    global delta_O,delta_h,E,O,h,w_output

    for i in range(len(delta_O)):
          #derivative of softmax is x(1-x)
          delta_O[i]=E[i] * (O[i]*(1-O[i]))

    for i in range(len(h)):
        val=0.0
        for j in range(len(delta_O)):
            #using derivative of relu i.e if x>0 then 1 else 0
            if h[i]>0:
                val+=delta_O[j]*(1)*w_output[i][j]
            else:
                val+=delta_O[j]*(0)*w_output[i][j]
        delta_h[i]=val



In [6]:
#Update weights

def update_weights():
    global learning_rate,w_hidden,w_output,delta_h,X,h,delta_O
    #update hidden layer weights
    for i in range(4):
        for j in range(5):
            w_hidden[i][j]+=(learning_rate*delta_h[j]*X[i])


    #update output layer weights
    for i in range(5):
        for j in range(3):
            w_output[i][j]+=(learning_rate*delta_O[j]*h[i])


In [7]:
#Update biases

def update_bias():
    global b_hidden,b_output,learning_rate,delta_h,delta_O

    #update hidden layer bias
    for i in range(len(b_hidden)):
        b_hidden[i]+=(learning_rate*delta_h[i])

    #update output layer bias
    for i in range(len(b_output)):
        b_output[i]+=(learning_rate*delta_O[i])


In [8]:
def backward_prop():
    func1()
    update_weights()
    update_bias()

In [ ]:
#***********************************************************
def train_MLP(X_train, y_train):
    global X, actual_output, E, delta_O, delta_h

    for i in range(len(X_train)):
        # Set input layer to current sample
        X = X_train[i]
        # Set actual output based on the label of the current sample
        if y_train[i] == 'Iris-setosa':
            actual_output = [1, 0, 0]
        elif y_train[i] == 'Iris-versicolor':
            actual_output = [0, 1, 0]
        elif y_train[i] == 'Iris-virginica':
            actual_output = [0, 0, 1]

        # Forward propagation
        forward_prop()

        # Calculate absolute error
        abs_error()

        # Backward propagation
        backward_prop()

#***********************************************************
def test_MLP(X_test,y_test):
    global X,E
    sum_loss=[0,0,0]
    correct_predictions=0
    for i in range(len(X_test)):
        # Set input layer to current sample
        X = X_test[i]

        # Forward propagation
        forward_prop()

        abs_error()
        #add error
        for i in range(len(sum_loss)):
          sum_loss[i]+=E[i]

        #Check result
        if (y_train[i] == 'Iris-setosa' and O[0]==max(O)) or (y_train[i] == 'Iris-versicolor' and O[1]==max(O)) or (y_train[i] == 'Iris-virginica' and O[2]==max(O)):
            correct_predictions+=1
    return correct_predictions,sum_loss


#***********************************************************
df=pd.read_csv('Iris.csv')
X = df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]  # Features
y = df['Species']  # Label

scaler = StandardScaler()

# Fit and transform the data
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

y_train = y_train.values.tolist()
y_test = y_test.values.tolist()


#***********************************************************
import random
#---------------------------------------
#Setting input layer
X=[1,1,1,1]  #since IRIS has 4 input columns

#Setting hidden layer
h=[0,0,0,0,0]

#Setting output layer
O=[0,0,0]  #since IRIS has 3 classes O[0]=Setosa,O[1]=Versicolour,O[2]=Virginica

#Setting weights between input and hidden layer

w_hidden=[]
for i in range(4):
    row=[]
    for j in range(5):
        row.append(random.uniform(0,1))
    w_hidden.append(row)


#Setting weights between hidden and output layer

w_output=[]

for i in range(5):
    row=[]
    for j in range(3):
        row.append(random.uniform(0,1))
    w_output.append(row)

#Setting hidden layer bias
b_hidden=[]
for i in range(5):
    b_hidden.append(random.uniform(0,1))

#Setting output layer bias
b_output=[]
for i in range(3):
    b_output.append(random.uniform(0,1))

# One-hot encoded label
actual_output = [0, 0, 0]

#Absolute error
E=[0,0,0]

delta_O=[0,0,0]
delta_h=[0,0,0,0,0]

learning_rate=0.001


#****************************************************************

# Training loop
iterations = 1000
for i in range(iterations):
    train_MLP(X_train, y_train)

#***********************************************************
correct_predictions,sum_loss=test_MLP(X_test,y_test)
print(correct_predictions,' out of ',len(X_test),' correct.')
accuracy=correct_predictions/len(X_test)
print('Accuracy=',round(accuracy*100,2),'%')
print('loss=')
for i in range(len(sum_loss)):
  print(round(sum_loss[i]/len(X_test),3))

29  out of  30  correct.
Accuracy= 96.67 %
loss=
0.008
0.954
0.962


In [99]:
#***************Using different range for weights********************************************
#Using IRIS dataset
def train_MLP(X_train, y_train):
    global X

    for i in range(len(X_train)):
        # Set input layer to current sample
        X = X_train[i]
        # Set actual output based on the label of the current sample
        if y_train[i] == 'Iris-setosa':
            actual_output = [1, 0, 0]
        elif y_train[i] == 'Iris-versicolor':
            actual_output = [0, 1, 0]
        elif y_train[i] == 'Iris-virginica':
            actual_output = [0, 0, 1]

        # Forward propagation
        forward_prop()

        # Calculate absolute error
        abs_error()

        # Backward propagation
        backward_prop()

#***********************************************************
def test_MLP(X_test,y_test):
    global X,E
    sum_loss=[0,0,0]
    correct_predictions=0
    for i in range(len(X_test)):
        # Set input layer to current sample
        X = X_test[i]

        # Forward propagation
        forward_prop()

        abs_error()
        #add error
        for i in range(len(sum_loss)):
          sum_loss[i]+=E[i]

        #Check result
        if (y_train[i] == 'Iris-setosa' and O[0]==max(O)) or (y_train[i] == 'Iris-versicolor' and O[1]==max(O)) or (y_train[i] == 'Iris-virginica' and O[2]==max(O)):
            correct_predictions+=1
    return correct_predictions,sum_loss


#***********************************************************
df=pd.read_csv('Iris.csv')
X = df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]  # Features
y = df['Species']  # Label

scaler = StandardScaler()

# Fit and transform the data
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

y_train = y_train.values.tolist()
y_test = y_test.values.tolist()


#***********************************************************
import random
#---------------------------------------
#Setting input layer
X=[1,1,1,1]  #since IRIS has 4 input columns

#Setting hidden layer
h=[0,0,0,0,0]

#Setting output layer
O=[0,0,0]  #since IRIS has 3 classes O[0]=Setosa,O[1]=Versicolour,O[2]=Virginica

#Setting weights between input and hidden layer

w_hidden=[]
for i in range(4):
    row=[]
    for j in range(5):
        row.append(random.uniform(-1,1))
    w_hidden.append(row)


#Setting weights between hidden and output layer

w_output=[]

for i in range(5):
    row=[]
    for j in range(3):
        row.append(random.uniform(-1,1))
    w_output.append(row)

#Setting hidden layer bias
b_hidden=[]
for i in range(5):
    b_hidden.append(random.uniform(-1,1))

#Setting output layer bias
b_output=[]
for i in range(3):
    b_output.append(random.uniform(-1,1))

# One-hot encoded label
actual_output = [0, 0, 0]

#Absolute error
loss = 0.0

E=[0,0,0]

delta_O=[0,0,0]
delta_h=[0,0,0,0,0]

learning_rate=0.001

#****************************************************************

# Training loop
iterations = 1000
for i in range(iterations):
    train_MLP(X_train, y_train)

#***********************************************************
correct_predictions,sum_loss=test_MLP(X_test,y_test)
print(correct_predictions,' out of ',len(X_test),' correct.')
accuracy=correct_predictions/len(X_test)
print('Accuracy=',round(accuracy*100,2),'%')
print('loss=')
for i in range(len(sum_loss)):
  print(round(sum_loss[i]/len(X_test),3))

26  out of  30  correct.
Accuracy= 86.67 %
loss=
0.146
0.854
0.0
